In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold,train_test_split,KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

#LOAD DATA
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')


In [ ]:
train_df.head()
train_df.info() #We have Sex, Ticket, Cabin, Embarked as categorical
train_df.describe(include='all')

In [ ]:
pd.isnull(train_df).sum() #Identify null values in training dataset
#pd.isnull(test_df).sum()

In [ ]:
df1 = train_df.groupby(['Survived','Sex'])['Survived'].count().unstack('Sex') #There is good chance females survive
df1.plot(kind='bar',stacked=True,)

In [ ]:
train_df['Ticket'].head(10) #Not a useful measure.
train_df.describe(include='all')

In [ ]:
train_df[['Embarked','Survived']].groupby('Embarked')['Survived'].value_counts().unstack('Survived').plot(kind = 'bar',stacked=True)

In [ ]:
train_df[['Pclass','Survived']].groupby('Pclass')['Survived'].value_counts().unstack('Survived').plot(kind='bar',stacked=True)

In [ ]:
train_df.corr()
sns.pairplot(train_df)

    **Feature Engineering and Visualization**

Looks like Srivived, Pclass, SibSp, Parch are categorical

In [ ]:
combine = [train_df,test_df]

for data in combine:
    for i in range(0,len(data)):
        data['Name'].iloc[i] = re.search('([a-zA-Z]+)\.',data['Name'].iloc[i]).group(0)
    data['Name'] = data['Name'].replace(r'\.','',regex=True)

        

In [ ]:
train_df['Name'].value_counts()
pd.crosstab(train_df['Name'],train_df['Sex'])

In [ ]:
combine = [train_df,test_df]
for data in combine: 
    data['Name'] = data['Name'].replace(['Capt','Col','Don','Dr','Jonkheer','Lady','Major','Rev','Sir','Countess'],'Others')
    data['Name'] = data['Name'].replace(['Mlle','Ms'],'Miss')
    data['Name'] = data['Name'].replace(['Mme'],'Mrs')


In [ ]:
train_df[['Name','Survived']].groupby('Name').mean()*100 #Average Survival Rate

In [ ]:
#plt.barh(train_df['Parch'].value_counts().index,train_df['Parch'].value_counts().values)
train_df['Parch'].value_counts().plot(kind='barh') #People traveling alone are more, hence there is higher chance of survivng

In [ ]:
train_df['SibSp'].value_counts()  
train_df[['SibSp','Survived']].groupby('SibSp')['Survived'].value_counts().unstack('Survived').plot(kind='bar',stacked='True')

In [ ]:
combine = [train_df,test_df]
for data in combine: 
    data['isAlone'] = 0
    for i in range(len(data)):
        if data['SibSp'].iloc[i] > 0 or data['Parch'].iloc[i] > 0:
            data['isAlone'].iloc[i] = 1
    data['FamSize'] = data['SibSp'] + data['Parch'] + 1

In [ ]:
train_df['Cabin'].value_counts() #Come back later and see if we can get anything out of cabin 

In [ ]:
train_df[['Fare','Survived']].groupby('Survived')['Fare'].mean()

In [ ]:
print(train_df[['Fare','Pclass']].groupby('Pclass')['Fare'].mean())
print(train_df[['Fare','Pclass']].groupby('Pclass')['Fare'].median())

train_df[['Fare','Pclass']].boxplot(by='Pclass')#Data with fare>200 seems to be like outliers

#train_df['FareClass'] = pd.qcut(train_df['Fare'],4)
#train_df['FareClass'].value_counts()

#train_df[train_df['Fare']>=200].count() #There are 20 such data points and they must be millionaries dropping them
#test_df[train_df['Fare']>=200].count()

train_df = train_df[train_df['Fare']<200] #In fare there are outliers. Eliminating those data points
test_df[test_df['Fare']>200]['Fare'] = train_df[train_df['Pclass']==1]['Fare'].mean() 



In [ ]:
train_df[['Fare','Pclass']].boxplot(by='Pclass')#Data with fare>200 seems to be like outliers

In [ ]:
pd.isnull(train_df['Fare']).sum()
pd.isnull(test_df['Fare']).sum()

test_df['Fare'] = test_df['Fare'].apply(lambda x: train_df['Fare'].mean() if pd.isna(x) else x)

In [ ]:
combine = [train_df,test_df]

for data in combine:
    data['Fare1'] = 0
    for i in range(len(data)):
        #print(data['Fare'].iloc[i])
        #if data['Fare'].iloc[i] <= 12.5:
        #    data['Fare1'].iloc[i] = 0
        #elif data['Fare'].iloc[i] > 12.5 and data['Fare'].iloc[i] <=25:
        #    data['Fare1'].iloc[i] = 1
        #else:
        #    data['Fare1'].iloc[i] = 2
        #print(data['Fare1'].iloc[i])

        if data['Fare'].iloc[i] <= 7.91:
            data['Fare1'].iloc[i] = 0
        elif data['Fare'].iloc[i] > 7.91 and data['Fare'].iloc[i] <= 14.45:
            data['Fare1'].iloc[i] = 1
        elif data['Fare'].iloc[i] > 14.45 and data['Fare'].iloc[i] <= 31:
            data['Fare1'].iloc[i] = 2
        else:
            data['Fare1'].iloc[i] = 3
#train_df['Fare'].value_counts()


In [ ]:
pd.isna(train_df).sum() #There are missing values in age and embarked

combine = [train_df, test_df]
for data in combine:    
    data['Age'] = data['Age'].apply(lambda x: train_df['Age'].mean() if pd.isna(x) else x)
    data['Embarked'] = data['Embarked'].apply(lambda x: train_df['Embarked'].mode()[0] if pd.isna(x) else x)


In [ ]:
train_df['Age'].hist()
#train_df['AgeGroup'] = pd.cut(train_df['Age'],5)
#train_df['AgeGroup'].value_counts()

#combine = [train_df,test_df]

#pd.isna(test_df['AgeGroup']).sum()
#train_df[pd.isna(train_df['AgeGroup'])]
    
bins = [0,16,32,48,64,np.inf]
#labels = ['Baby','Teens','Youth','YoungAdults','Adults']
#train_df['AgeGroup'] = pd.cut(train_df['Age'],bins,labels=labels)
#test_df['AgeGroup'] = pd.cut(test_df['Age'],bins,labels=labels)

#bins = [0,10,20,30,50,np.inf]
labels = ['Child','Youth','Young Adults','Adults','Old']
train_df['AgeGroup'] = pd.cut(train_df['Age'],bins,labels=labels)

test_df['AgeGroup'] = pd.cut(test_df['Age'],bins,labels=labels)


In [ ]:
plt.subplot(1,2,1)
plt.title('Survived = 0')
train_df[train_df['Survived']==0]['Fare'].hist()
plt.subplot(1,2,2)
plt.title('Survived = 1')
train_df[train_df['Survived']==1]['Fare'].hist()

Catergorical Variable creation 

In [ ]:
sex_map = {'male':1,'female':0}
#train_df1 = train_df.copy()
train_df['Sex'] = train_df['Sex'].map(sex_map)
test_df['Sex'] =test_df['Sex'].map(sex_map)

In [ ]:
pd.isna(train_df['Sex']).sum() #To check if there are any NUll values present
embark_map = {'C':2,'Q':1, 'S':0}
train_df['Embarked'] = train_df['Embarked'].map(embark_map)
test_df['Embarked'] = test_df['Embarked'].map(embark_map)

In [ ]:
Label = LabelEncoder()
train_df['Name']= Label.fit_transform(train_df['Name'])
test_df['Name']= Label.fit_transform(test_df['Name'])

In [ ]:
#Age_label = LabelEncoder()
#pd.isna(train_df['AgeGroup']).sum()

#train_df['AgeGroup'] = Age_label.fit_transform(train_df['AgeGroup'])
#train_df['AgeGroup'].value_counts()
#test_df['AgeGroup'] = Age_label.fit_transform(test_df['AgeGroup'])

#train_df['Age'] = train_df['AgeGroup']*train_df['Pclass']
#test_df['Age'] = test_df['AgeGroup']*test_df['Pclass']

#train_df['Age'].value_counts() #What happened to 5 & 11
train_df['AgeGroup'].value_counts()

In [ ]:
combine = [train_df,test_df]
for data in combine:
    for i in range(len(data)):
        if data['AgeGroup'].iloc[i] == 'Child':
            data['Age'].iloc[i] = 0
        elif data['AgeGroup'].iloc[i] == 'Youth':
            data['Age'].iloc[i] = 1
        elif data['AgeGroup'].iloc[i] == 'Young Adults':
            data['Age'].iloc[i] = 2
        elif data['AgeGroup'].iloc[i] == 'Adults':
            data['Age'].iloc[i] = 3
        else:
            data['Age'].iloc[i] = 4


In [ ]:
#train_df['AgeGroup'].value_counts()
train_df.drop(['Ticket','Cabin','PassengerId','Fare','SibSp','Parch','AgeGroup','Pclass'],axis=1,inplace=True)
test_df.drop(['Ticket','Cabin','Fare','SibSp','Parch','AgeGroup','Pclass'],axis=1,inplace=True)

#train_df.drop(['FareClass'],axis=1,inplace=True)
#test_df.drop(['FareClass'],axis=1,inplace=True)

**Building models**

In [ ]:
train_df.head() #Try converting categorical feature columns to objects

In [ ]:
#Split = StratifiedKFold()
target = train_df['Survived']
Input = train_df.drop(['Survived'],axis=1)
x_train,x_val,y_train,y_val = train_test_split(Input,target,random_state = 1000, test_size = 0.33)

LOGISTIC REGRESSION

In [ ]:
log_model = LogisticRegression()
log_model.fit(x_train,y_train)
y_pred1 = log_model.predict(x_val)
acc_log = round(accuracy_score(y_pred1,y_val)*100,2)
acc_log

In [ ]:
plt.barh(x_train.columns.values,(np.round((log_model.coef_),2))[0])

RANDOM FOREST CLASSIFIER

In [ ]:
rand_model = RandomForestClassifier()
rand_model.fit(x_train,y_train)
y_pred2 = rand_model.predict(x_val)
acc_rand = round(accuracy_score(y_pred2,y_val)*100,4)
print("Accuracy Calculation",acc_rand)
round(rand_model.score(x_train,y_train),4)*100

In [ ]:
col = pd.Series(x_train.columns.values)
imp = pd.Series(rand_model.feature_importances_)
features = pd.concat([col,imp],axis=1,keys=['Features','Values'])
plt.barh(features['Features'],features['Values'])

**DECISION TREES**

In [ ]:
DT_model = DecisionTreeClassifier()
DT_model.fit(x_train,y_train)
y_pred3 = DT_model.predict(x_val)
acc_DT = np.round(accuracy_score(y_pred3,y_val),4)*100
acc_DT
#DT_model.score(x_train,y_train)

**KNN Classifier**

In [ ]:
knn_model = KNeighborsClassifier()
knn_model.fit(x_train,y_train)
y_pred4 = knn_model.predict(x_val)
acc_knn = np.round(accuracy_score(y_pred4,y_val),4)*100
acc_knn

**Support Vector Machines**

In [ ]:
suppv = SVC()
suppv.fit(x_train,y_train)
y_pred5 = suppv.predict(x_test)
acc_sv = round(accuracy_score(y_pred5,y_test)*100,2)
acc_sv

**Naive Bayes Classifier**

In [ ]:
nb_mod = GaussianNB()
nb_mod.fit(x_train,y_train)
y_pred6 = nb_mod.predict(x_test)
acc_nb = round(accuracy_score(y_pred6,y_test)*100,2)
acc_nb

In [ ]:
model_accuracies = pd.DataFrame({'Model':['Logistic Regression','Random Forests','Decision Tree','Knn Classifier','SVM','Naive Bayes'],
                                 'Accuracy':[acc_log,acc_rand,acc_DT,acc_knn,acc_sv,acc_nb]})
model_accuracies.sort_values(by='Accuracy',ascending=False)

In [ ]:
y_sub1 = rand_model.predict(test_df.iloc[:,1:])
y_sub2 = suppv.predict(test_df.iloc[:,1:])

submission = pd.DataFrame({'PassengerId':test_df['PassengerId'],'Survived':y_sub1})
submission.to_csv('./submission_rand.csv', index=False)

submission = pd.DataFrame({'PassengerId':test_df['PassengerId'],'Survived':y_sub2})
submission.to_csv('./submission_supvec.csv', index=False)


In [ ]:
Kf = KFold(n_splits=3,random_state=1000,shuffle=False)
target = train_df['Survived']
train_df.drop(['Survived'],axis=1,inplace=True)

mod_acc = []
mod_features = []

for train_index, test_index in Kf.split(train_df):
    x_train, y_train = train_df[train_index[0]:train_index[-1]], target[train_index[0]:train_index[-1]]
    x_test, y_test = train_df[test_index[0]:test_index[-1]], target[test_index[0]:test_index[-1]]
    randForest = RandomForestClassifier()
    randForest.fit(x_train,y_train)
    y_pred = randForest.predict(x_test)
    mod_acc.append(round(accuracy_score(y_pred,y_test)*100,4))
    mod_features.append(randForest.feature_importances_)

for i in range(len(mod_features)):    
    col = pd.Series(x_train.columns.values)
    imp = pd.Series(mod_features[i])
    features = pd.concat([col,imp],axis=1,keys=['Features','Values'])
    plt.subplot(1,3,(i+1))
    plt.barh(features['Features'],features['Values'])

**Comparing all the models**